In [1]:
import time
import numpy as np
import pandas as pd
from sklearn.ensemble import RandomForestRegressor
#from sklearn import pipeline, model_selection
from sklearn import pipeline, grid_search
#from sklearn.feature_extraction import DictVectorizer
from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.pipeline import FeatureUnion
from sklearn.decomposition import TruncatedSVD
#from sklearn.feature_extraction.text import CountVectorizer, TfidfTransformer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics import mean_squared_error, make_scorer
#from nltk.metrics import edit_distance
from nltk.stem.porter import *
stemmer = PorterStemmer()
#from nltk.stem.snowball import SnowballStemmer #0.003 improvement but takes twice as long as PorterStemmer
#stemmer = SnowballStemmer('english')
import re
#import enchant
import random
random.seed(2016)
import string
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer

#import enchant



In [2]:
df_train = pd.read_excel('pre_processed_data\df_train.xlsx')
df_test = pd.read_excel('pre_processed_data\df_test.xlsx')

In [3]:
df_prod = pd.read_excel('pre_processed_data\df_prod.xlsx')
df_brand = pd.read_excel('pre_processed_data\df_brand.xlsx')
df_attr = pd.read_excel('pre_processed_data\df_attr.xlsx')

## Compute scores

In [156]:
def num_prod_title_score(uid, search_term):
    if isinstance(search_term, str):
        prod_title_list = str(df_train[df_train['product_uid'] == uid ][0:1]['product_title_num'].values[0]).split()
        total_len = len(search_term.split())
        count = 0        
        for word in search_term.split():
            count += prod_title_list.count(word)                
        if total_len != 0:
            
            return count/total_len
        else:
            return 0
    else:
        return 0
    
def str_prod_title_score(uid, search_term):
    if isinstance(search_term, str):
        prod_title_list = str(df_train[df_train['product_uid'] == uid ][0:1]['product_title_str'].values[0]).split()
        total_len = len(search_term.split())
        count = 0
        for word in search_term.split():
            count += prod_title_list.count(word)                
        if total_len != 0:
            return count/total_len
        else:
            return 0
    else:
        return 0

In [157]:
df_train['f_num_prod_title_score'] = df_train.apply(lambda df_train: num_prod_title_score(df_train['product_uid'],df_train['search_term_num'] ), axis = 1)
df_train['f_str_prod_title_score'] = df_train.apply(lambda df_train: str_prod_title_score(df_train['product_uid'],df_train['search_term_str'] ) ,axis = 1)

In [70]:
def num_prod_title_score_test(uid, search_term):
    if isinstance(search_term, str):
        prod_title_list = str(df_test[df_test['product_uid'] == uid ][0:1]['product_title_num'].values[0]).split()
        total_len = len(search_term.split())
        count = 0
        for word in search_term.split():
            count += prod_title_list.count(word)                
        if total_len != 0:            
            return count/total_len
        else:
            return 0
    else:
        return 0
    
def str_prod_title_score_test(uid, search_term):    
    if isinstance(search_term, str):
        prod_title_list = str(df_test[df_test['product_uid'] == uid ][0:1]['product_title_str'].values[0]).split()
        total_len = len(search_term.split())
        #print ("search term:",search_term,"product_title_str" ,prod_title_list) #
        count = 0
        for word in search_term.split():
            count += prod_title_list.count(word)                
        if total_len != 0:
            return count/total_len
        else:
            return 0
    else:
        return 0

In [71]:
df_test['f_num_prod_title_score'] = df_test.apply(lambda df_test: num_prod_title_score_test(df_test['product_uid'],df_test['search_term_num'] ), axis = 1)
df_test['f_str_prod_title_score'] = df_test.apply(lambda df_test: str_prod_title_score_test(df_test['product_uid'],df_test['search_term_str'] ) ,axis = 1)

In [73]:
def prod_brand_score(uid, search_term):
    if isinstance(search_term, str):
        if any(df_brand['product_uid'] == uid) is False:
            return 0
        prod_title_list = df_brand[df_brand['product_uid'] == uid ][0:1]['brand'].values[0].split()
        #print prod_title_list
        search_term_list = search_term.split()
        total_len = len(search_term_list)
        count = 0
        for word in search_term.split():
            count += prod_title_list.count(word)    
        if total_len != 0:
            return count/total_len
        else:
            return 0
    else:
        return 0

df_train['f_str_prod_brand_score'] = df_train.apply(lambda df_train: prod_brand_score(df_train['product_uid'],df_train['search_term_str'] ) ,axis = 1)
df_test['f_str_prod_brand_score'] = df_test.apply(lambda df_test: prod_brand_score(df_test['product_uid'],df_test['search_term_str'] ) ,axis = 1)

In [92]:
def num_prod_desc_score(uid, search_term):    
    prod_title_list = str(df_prod[df_prod['product_uid'] == uid ][0:1]['product_description_num'].values[0]).split()
    search_term_list = str(search_term).split()
    total_len = len(search_term_list)
    count = 0
    for word in str(search_term).split():
        count += prod_title_list.count(str(word))    
    if total_len != 0:
        return count/total_len
    else:
        return 0

    
def str_prod_desc_score(uid, search_term):
    if isinstance(search_term, str):
        prod_title_list = df_prod[df_prod['product_uid'] == uid ][0:1]['product_description_str'].values[0].split()
        search_term_list = search_term.split()
        total_len = len(search_term_list)
        count = 0
        for word in search_term.split():
            count += prod_title_list.count(word)    
        if total_len != 0:
            return count/total_len
        else:
            return 0
    else:
        return 0
    

In [93]:
df_train['f_str_prod_desc'] = df_train.apply(lambda df_train: str_prod_desc_score(df_train['product_uid'],df_train['search_term_str'] ) ,axis = 1)
df_train['f_num_prod_desc'] = df_train.apply(lambda df_train: num_prod_desc_score(df_train['product_uid'],df_train['search_term_num'] ) ,axis = 1)

In [94]:
df_test['f_str_prod_desc'] = df_test.apply(lambda df_test: str_prod_desc_score(df_test['product_uid'],df_test['search_term_str'] ) ,axis = 1)
df_test['f_num_prod_desc'] = df_test.apply(lambda df_test: num_prod_desc_score(df_test['product_uid'],df_test['search_term_num'] ) ,axis = 1)

In [130]:
def str_prod_attr_score(uid, search_term):
    if isinstance(search_term, str): 
        l1 = []
        for a in df_attr[df_attr['product_uid'] == 100001 ][df_attr['name'] != "MFG Brand Name	"]['value_str'].iteritems():
            l1 += str(a[1]).split()
        l2 = [a for a in l1 if a != 'nan']
        search_term_list = search_term.split()
        total_len = len(search_term_list)
        count = 0
        for word in search_term.split():
            count += l2.count(word)    
        if total_len != 0:
            return count/total_len
        else:
            return 0
    else:
        return 0
    
def num_prod_attr_score(uid, search_term):
    if isinstance(search_term, str): 
        l1 = []
        for a in df_attr[df_attr['product_uid'] == 100001 ][df_attr['name'] != "MFG Brand Name	"]['value_num'].iteritems():
            l1 += str(a[1]).split()
        l2 = [a for a in l1 if a != 'nan']
        search_term_list = search_term.split()
        total_len = len(search_term_list)
        count = 0
        for word in search_term.split():
            count += l2.count(word)    
        if total_len != 0:
            return count/total_len
        else:
            return 0
    else:
        return 0
               


In [145]:
df_train['f_num_attr'] = df_train.apply(lambda df_train: num_prod_desc_score(df_train['product_uid'],df_train['search_term_num'] ) ,axis = 1)
df_train['f_str_attr'] = df_train.apply(lambda df_train: str_prod_desc_score(df_train['product_uid'],df_train['search_term_str'] ) ,axis = 1)

In [146]:
df_test['f_num_attr'] = df_test.apply(lambda df_test: num_prod_desc_score(df_test['product_uid'],df_test['search_term_num'] ) ,axis = 1)
df_test['f_str_attr'] = df_test.apply(lambda df_test: str_prod_desc_score(df_test['product_uid'],df_test['search_term_str'] ) ,axis = 1)

In [154]:
df_test.head(1)

,id,product_uid,product_title,search_term,search_term_num,search_term_str,product_title_num,product_title_str,f_num_prod_title_score,f_str_prod_title_score,f_str_prod_brand_score,f_str_prod_desc,f_num_prod_desc,f_num_attr,f_str_attr
0,1,100001,simpson strong tie 12 gaug angl,90deg. bracket,NaN,90deg. bracket,12.0,simpson strong tie gaug angl,0,0,0,0,0,0,0


In [125]:
train_X = df_train[['f_num_prod_title_score','f_str_prod_title_score','f_str_prod_brand_score','f_num_prod_desc','f_str_prod_desc1','f_str_attr1']]
train_Y = df_train[['relevance']]

test_X = df_test[['f_num_prod_title_score','f_str_prod_title_score','f_str_prod_brand_score','f_num_prod_desc','f_str_prod_desc1','f_str_attr1']]


In [126]:
from sklearn.ensemble import RandomForestRegressor
#from sklearn.svm import SVR
clf = RandomForestRegressor(n_estimators=500,random_state = 2016)
clf.fit(train_X, np.array(train_Y['relevance'].values.tolist()))
out = (clf.predict(test_X))
df_out = pd.DataFrame(columns=['id','relevance'])
df2 = pd.DataFrame({'id': [1], 'relevance': 2})
df_out = df_out.append(df2)

df_r1 = df_test[['id']]
df_r2 = pd.DataFrame(out, columns=['relevance'])
result = pd.concat([df_r1, df_r2], axis=1)



In [127]:
result[result.relevance > 3] 

,id,relevance


In [128]:
result.to_csv("results10.csv")

In [120]:
result[result.relevance < 1]

,id,relevance


In [176]:
df_train.to_excel('pre_processed_data\df_train.xlsx')
df_test.to_excel('pre_processed_data\df_test.xlsx')

## TF-IDF

In [24]:
from sklearn.feature_extraction.text import CountVectorizer

In [10]:
df_prod.head()

,product_uid,product_description,product_description_num,product_description_str
0,100001,angl make joint stronger also provid consist s...,90.0 9.0 3.0 1.0 12.0 10.0 1.0 1.0,angl make joint stronger also provid consist s...
1,100002,behr premium textur deckov innov solid color c...,2.0,behr premium textur deckov innov solid color c...
2,100003,classic architectur meet contemporari design e...,124.1 2.0 45.1,classic architectur meet contemporari design e...
3,100004,grape solar 265watt. polycrystallin pv solar p...,4.0 4.0,grape solar 265watt. polycrystallin pv solar p...
4,100005,updat bathroom delta vero singl handl shower f...,3.0 10000.0,updat bathroom delta vero singl handl shower f...


In [29]:
bow_transformer = CountVectorizer()
bow_transformer.fit(df_prod['product_description'])

CountVectorizer(analyzer='word', binary=False, decode_error='strict',
        dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
        lowercase=True, max_df=1.0, max_features=None, min_df=1,
        ngram_range=(1, 1), preprocessor=None, stop_words=None,
        strip_accents=None, token_pattern='(?u)\\b\\w\\w+\\b',
        tokenizer=None, vocabulary=None)

In [80]:
test=df_prod['product_description'][1]
a = bow_transformer.transform(test.split())


In [55]:
print (bow_transformer.get_feature_names()[16870])

angl


In [59]:
messages_bow = bow_transformer.transform(df_prod['product_description'])

In [61]:
print ('Shape of Sparse Matrix: ', messages_bow.shape)
print ('Amount of Non-Zero occurences: ', messages_bow.nnz)
print ('sparsity: %.2f%%' % (100.0 * messages_bow.nnz / (messages_bow.shape[0] * messages_bow.shape[1])))

Shape of Sparse Matrix:  (124428, 53604)
Amount of Non-Zero occurences:  8339983
sparsity: 0.13%


In [62]:
from sklearn.feature_extraction.text import TfidfTransformer


In [72]:
tfidf_transformer = TfidfTransformer().fit(messages_bow)

In [81]:
tfidf4 = tfidf_transformer.transform(a)
for a in tfidf4:
    print (a)

  (0, 18573)	1.0
  (0, 41292)	1.0
  (0, 48515)	1.0
  (0, 23857)	1.0
  (0, 32815)	1.0
  (0, 46169)	1.0
  (0, 22065)	1.0
  (0, 21921)	1.0
  (0, 19637)	1.0
  (0, 38803)	1.0
  (0, 52203)	1.0
  (0, 52831)	1.0
  (0, 22270)	1.0
  (0, 17950)	1.0
  (0, 34936)	1.0
  (0, 16111)	1.0
  (0, 1118)	1.0
  (0, 15934)	1.0
  (0, 43104)	1.0
  (0, 28385)	1.0
  (0, 22945)	1.0
  (0, 25410)	1.0
  (0, 21921)	1.0
  (0, 48946)	1.0
  (0, 52950)	1.0
  (0, 23848)	1.0
  (0, 42866)	1.0
  (0, 52517)	1.0
  (0, 38049)	1.0
  (0, 35361)	1.0
  (0, 18708)	1.0
  (0, 43167)	1.0
  (0, 47719)	1.0
  (0, 41563)	1.0
  (0, 41304)	1.0
  (0, 47731)	1.0
  (0, 50924)	1.0
  (0, 17095)	1.0
  (0, 18573)	1.0
  (0, 41482)	1.0
  (0, 24640)	1.0
  (0, 20246)	1.0
  (0, 43099)	1.0
  (0, 44837)	0.555528900722
  (0, 41571)	0.588118143822
  (0, 37863)	0.587796469341
  (0, 12092)	1.0
  (0, 28367)	1.0
  (0, 43246)	1.0
  (0, 52831)	1.0
  (0, 22221)	1.0
  (0, 23848)	1.0
  (0, 42327)	1.0
  (0, 40966)	1.0
  (0, 19169)	1.0
  (0, 24810)	1.0
  (0, 16622)	1.0

In [78]:
messages_bow

<124428x53604 sparse matrix of type '<class 'numpy.int64'>'
	with 8339983 stored elements in Compressed Sparse Row format>

In [105]:
print (tfidf_transformer.idf_[bow_transformer.vocabulary_['product']])

2.50646312538


In [88]:
df_prod['product_description'][1]

'behr premium textur deckov innov solid color coat bring old weather wood concret back life advanc 100% acryl resin formula creat durabl coat tire worn deck rejuven whole new look best result sure properli prepar surfac use applic behr product display california residents: see&nbsp;proposit 65in. format reviv wood composit deck rail porch boat dock also great concret pool deck patio sidewalk 100% acryl solid color coat resist crack peel conceal splinter crack 1/4in. provid durabl mildew resist finish cover 75sq.ft. 2 coat per gallon creat textur slip resist finish best result prepar appropri behr product wood concret surfac actual paint color may vari screen printer repres color avail tint store onlin price includ paint care fee follow states: ca co ct mn ri vt'

In [97]:
bow_transformer.vocabulary_['tint']

48934

In [123]:
def str_prod_desc_score1(uid, search_term):
    if isinstance(search_term, str):
        prod_title_list = df_prod[df_prod['product_uid'] == uid ][0:1]['product_description_str'].values[0].split()
        search_term_list = search_term.split()
        total_len = len(search_term_list)
        count = 0
        for word in search_term.split():
            try:
                count += prod_title_list.count(word)  * tfidf_transformer.idf_[bow_transformer.vocabulary_[word]]
            except:
                count += prod_title_list.count(word)    
        return count
    else:
        return 0
def str_prod_attr_score1(uid, search_term):
    if isinstance(search_term, str): 
        l1 = []
        for a in df_attr[df_attr['product_uid'] == 100001 ][df_attr['name'] != "MFG Brand Name	"]['value_str'].iteritems():
            l1 += str(a[1]).split()
        l2 = [a for a in l1 if a != 'nan']
        search_term_list = search_term.split()
        total_len = len(search_term_list)
        count = 0
        for word in search_term.split():
            try:
                count += l2.count(word)     * tfidf_transformer.idf_[bow_transformer.vocabulary_[word]]
            except:
                count += l2.count(word)    
        return count
    else:
        return 0

In [124]:
df_train['f_str_prod_desc1'] = df_train.apply(lambda df_train: str_prod_desc_score1(df_train['product_uid'],df_train['search_term_str'] ) ,axis = 1)
df_test['f_str_prod_desc1'] = df_test.apply(lambda df_test: str_prod_desc_score1(df_test['product_uid'],df_test['search_term_str'] ) ,axis = 1)

df_train['f_str_attr1'] = df_train.apply(lambda df_train: str_prod_desc_score1(df_train['product_uid'],df_train['search_term_str'] ) ,axis = 1)
df_test['f_str_attr1'] = df_test.apply(lambda df_test: str_prod_desc_score1(df_test['product_uid'],df_test['search_term_str'] ) ,axis = 1)

In [110]:
df_test.head()

,id,product_uid,product_title,search_term,search_term_num,search_term_str,product_title_num,product_title_str,f_num_prod_title_score,f_str_prod_title_score,f_str_prod_brand_score,f_str_prod_desc,f_num_prod_desc,f_num_attr,f_str_attr,f_str_prod_desc1
0,1,100001,simpson strong tie 12 gaug angl,90deg. bracket,NaN,90deg. bracket,12.0,simpson strong tie gaug angl,0,0.000000,0.000000,0.000000,0,0,0.000000,0.000000
1,4,100001,simpson strong tie 12 gaug angl,metal l bracket,NaN,metal l bracket,12.0,simpson strong tie gaug angl,0,0.000000,0.000000,0.000000,0,0,0.000000,0.000000
2,5,100001,simpson strong tie 12 gaug angl,simpson sky abl,NaN,simpson sky abl,12.0,simpson strong tie gaug angl,0,0.333333,0.333333,0.333333,0,0,0.333333,2.231207
3,6,100001,simpson strong tie 12 gaug angl,simpson strong tie,NaN,simpson strong tie,12.0,simpson strong tie gaug angl,0,1.000000,0.333333,1.666667,0,0,1.666667,8.203344
4,7,100001,simpson strong tie 12 gaug angl,simpson strong tie hcc 668,668.0,simpson strong tie hcc,12.0,simpson strong tie gaug angl,0,0.750000,0.250000,1.250000,0,0,1.250000,6.152508


In [122]:
def str_prod_attr_score1(uid, search_term):
    if isinstance(search_term, str): 
        l1 = []
        for a in df_attr[df_attr['product_uid'] == 100001 ][df_attr['name'] != "MFG Brand Name	"]['value_str'].iteritems():
            l1 += str(a[1]).split()
        l2 = [a for a in l1 if a != 'nan']
        search_term_list = search_term.split()
        total_len = len(search_term_list)
        count = 0
        for word in search_term.split():
            try:
                count += l2.count(word)     * tfidf_transformer.idf_[bow_transformer.vocabulary_[word]]
            except:
                count += l2.count(word)    
        return count
    else:
        return 0

In [112]:
df_train['f_str_attr1'] = df_train.apply(lambda df_train: str_prod_desc_score1(df_train['product_uid'],df_train['search_term_str'] ) ,axis = 1)
df_test['f_str_attr1'] = df_test.apply(lambda df_test: str_prod_desc_score1(df_test['product_uid'],df_test['search_term_str'] ) ,axis = 1)

In [113]:
df_train.head()

,id,product_uid,product_title,search_term,relevance,search_term_num,search_term_str,product_title_num,product_title_str,f_str_prod_brand_score,f_str_prod_desc,f_num_prod_desc,f_num_attr,f_str_attr,f_num_prod_title_score,f_str_prod_title_score,f_str_prod_desc1,f_str_attr1
0,2,100001,simpson strong tie 12 gaug angl,angl bracket,3.00,NaN,angl bracket,12.0,simpson strong tie gaug angl,0,1.500000,0,0,1.500000,0,0.500000,6.737846,6.737846
1,3,100001,simpson strong tie 12 gaug angl,l bracket,2.50,NaN,l bracket,12.0,simpson strong tie gaug angl,0,0.000000,0,0,0.000000,0,0.000000,0.000000,0.000000
2,9,100002,behr premium textur deck 1gal. #sc 141 tugboat...,deck,3.00,NaN,deck,141.0,behr premium textur deck 1gal. #sc tugboat woo...,0,3.000000,0,0,3.000000,0,1.000000,12.617202,12.617202
3,16,100005,delta vero 1 handl shower faucet trim kit chro...,rain shower head,2.33,NaN,rain shower head,1.0,delta vero handl shower faucet trim kit chrome...,0,0.333333,0,0,0.333333,0,0.333333,1.384526,1.384526
4,17,100005,delta vero 1 handl shower faucet trim kit chro...,shower faucet,2.67,NaN,shower faucet,1.0,delta vero handl shower faucet trim kit chrome...,0,1.000000,0,0,1.000000,0,1.000000,4.035147,4.035147


In [1]:
df_train['f_str_prod_title_score'] = df_train.apply(lambda df_train: str_prod_title_score(df_train['product_uid'],df_train['search_term_str'] ) ,axis =